# 영화 관객수 예측
- 회귀모델을 사용해 영화 관객수 예측
- 데이터 이해, 분석 목표에 따른 처리 과정 연습

## 데이터 피쳐
- title : 영화의 제목
- distributor : 배급사
- genre : 장르
- release_time : 개봉일
- time : 상영시간(분)
- screening_rat : 상영등급
- director : 감독이름
- dir_prev_bfnum : 해당 감독이 이 영화를 만들기 전 제작에 참여한 영화에서의 평균 관객수(단 관객수가 알려지지 않은 영화 제외)
- dir_prev_num : 해당 감독이 이 영화를 만들기 전 제작에 참여한 영화의 개수(단 관객수가 알려지지 않은 영화 제외)
- num_staff : 스텝수
- num_actor : 주연배우수
- box_off_num : 관객수

In [1]:
import pandas as pd

In [4]:
train = pd.read_csv('movies_train.csv')
train.head()

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,NaN,0,91,2,23398
1,내부자들,(주)쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501
2,은밀하게 위대하게,(주)쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.25,4,343,4,6959083
3,나는 공무원이다,(주)NEW,코미디,2012-07-12,101,전체 관람가,구자홍,23894.00,2,20,6,217866
4,불량남녀,쇼박스(주)미디어플렉스,코미디,2010-11-04,108,15세 관람가,신근호,1.00,1,251,2,483387


In [5]:
test = pd.read_csv('movies_test.csv')
submission = pd.read_csv('submission.csv')

In [6]:
submission.head()

,title,box_off_num
0,용서는 없다,0
1,아빠가 여자를 좋아해,0
2,하모니,0
3,의형제,0
4,평행 이론,0


## 데이터 이해하기

In [11]:
#데이터의 길이 확인(행의 개수 확인)
len(train)

600

In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           600 non-null    object 
 1   distributor     600 non-null    object 
 2   genre           600 non-null    object 
 3   release_time    600 non-null    object 
 4   time            600 non-null    int64  
 5   screening_rat   600 non-null    object 
 6   director        600 non-null    object 
 7   dir_prev_bfnum  270 non-null    float64
 8   dir_prev_num    600 non-null    int64  
 9   num_staff       600 non-null    int64  
 10  num_actor       600 non-null    int64  
 11  box_off_num     600 non-null    int64  
dtypes: float64(1), int64(5), object(6)
memory usage: 56.4+ KB


### ndim 함수 : 데이터의 차원 수 확인
- Numpy 배열에서 차원수를 확인하는 함수
- 데이터셋이 이미지, 텍스트, 시계열 등 어떤 형태의 데이터인지 알수 있음

In [13]:
#데이터의 차원 확인(ndim)
train.ndim

2

### shape 함수 : 배열의 구조(크기, 형태)

In [14]:
train.shape

(600, 12)

### size 함수 : 데이터의 크기
- 데이터 요수 수(개수) 반환
- 전체 데이터의 개수 파악

In [15]:
train.size

7200

### 데이터의 feature 목록 확인
- columns 함수
- df.columns

In [17]:
train.columns
len(train.columns)

12

In [19]:
len(test.columns)

11

In [20]:
train['num_actor'].dtype

dtype('int64')

In [21]:
train['dir_prev_bfnum'].dtype

dtype('float64')

### 데이터의 기술통계량 확인
- describe() 함수
- 수치형 데이터로 이루어진 열들의 분포와 중심성, 흩어진 정도 등을 파악
- 결측값을 제외하고 통계량 계산

In [22]:
train.describe()

,time,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
count,600.000000,2.700000e+02,600.000000,600.000000,600.000000,6.000000e+02
mean,100.863333,1.050443e+06,0.876667,151.118333,3.706667,7.081818e+05
std,18.097528,1.791408e+06,1.183409,165.654671,2.446889,1.828006e+06
min,45.000000,1.000000e+00,0.000000,0.000000,0.000000,1.000000e+00
25%,89.000000,2.038000e+04,0.000000,17.000000,2.000000,1.297250e+03
50%,100.000000,4.784236e+05,0.000000,82.500000,3.000000,1.259100e+04
75%,114.000000,1.286569e+06,2.000000,264.000000,4.000000,4.798868e+05
max,180.000000,1.761531e+07,5.000000,869.000000,25.000000,1.426277e+07


### 결측치 확인
- isna(), isnull() : 결측값이 존재하면 true, 존재하지 않으면 false 반환
- notnull() : 위와 반대로 동작(결측값이 아니면 True)

In [23]:
train.isnull()

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
0,False,False,False,False,False,False,False,True,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
595,False,False,False,False,False,False,False,False,False,False,False,False
596,False,False,False,False,False,False,False,False,False,False,False,False
597,False,False,False,False,False,False,False,True,False,False,False,False
598,False,False,False,False,False,False,False,True,False,False,False,False


In [24]:
train.notnull()

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
0,True,True,True,True,True,True,True,False,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...
595,True,True,True,True,True,True,True,True,True,True,True,True
596,True,True,True,True,True,True,True,True,True,True,True,True
597,True,True,True,True,True,True,True,False,True,True,True,True
598,True,True,True,True,True,True,True,False,True,True,True,True


### 컬럼별 결측치 수 파악
- isnull().sum()

In [25]:
train.isnull().sum()

title               0
distributor         0
genre               0
release_time        0
time                0
screening_rat       0
director            0
dir_prev_bfnum    330
dir_prev_num        0
num_staff           0
num_actor           0
box_off_num         0
dtype: int64

### 결측치가 제거된 데이터셋 생성
- dropna() : 결측치가 포함된 행 삭제

In [28]:
train_dropna = train.dropna()
train_dropna.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 270 entries, 1 to 596
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           270 non-null    object 
 1   distributor     270 non-null    object 
 2   genre           270 non-null    object 
 3   release_time    270 non-null    object 
 4   time            270 non-null    int64  
 5   screening_rat   270 non-null    object 
 6   director        270 non-null    object 
 7   dir_prev_bfnum  270 non-null    float64
 8   dir_prev_num    270 non-null    int64  
 9   num_staff       270 non-null    int64  
 10  num_actor       270 non-null    int64  
 11  box_off_num     270 non-null    int64  
dtypes: float64(1), int64(5), object(6)
memory usage: 27.4+ KB


### 결측치 대채
- fillna()

In [29]:
train_fillna = train.fillna(0)

## 회귀분석
### 회귀분석을 위한 표현식 작성하기(formula)
- 프로젝트 목표 : submisssion 파일의 box_off_num을 예측값으로 채우기
- (유의) 문자형변수는 수치형으로 변환해야하는데, 이번 프로젝트에서는 수치형 데이터만 사용하여 회귀분석 모델을 구성해봄

In [30]:
formula = """
box_off_num ~ time + dir_prev_num + num_staff + num_actor
"""

### 영화관객 수 회귀분석 모델 정의 및 학습하기
- statsmodels 라이브러리 : 회귀분석 모델 생성
- sm.OLS.from_formula() 메서드를 사용하여 선형 회귀 모델을 생성
- OLS : ordinary least squares => 잔차제곱합(RSS) 최소화하는 방식

In [33]:
import statsmodels.api as sm

model = sm.OLS.from_formula(formula, data=train)
model = model.fit()    #회귀분석 모델을 주어진 데이터에 학습시키는 과정

### 영화 관객 수 모델 예측 결과 확인
- model 객체의 predict() 메소드 사용하여, Test 데이터셋 예측
- test 데이터셋의 각 행에 대한 예측값이 포함된 시리즈(Series)로 반환됨

In [34]:
predict = model.predict(test)
predict.head()

0    1.823796e+06
1    1.535341e+06
2    2.410155e+06
3    2.131345e+06
4    1.834060e+06
dtype: float64

### 예측된 영화 관객 수를 submission 파일에 추가

In [35]:
submission['box_off_num'] = predict
submission.head()

,title,box_off_num
0,용서는 없다,1.823796e+06
1,아빠가 여자를 좋아해,1.535341e+06
2,하모니,2.410155e+06
3,의형제,2.131345e+06
4,평행 이론,1.834060e+06


In [36]:
submission.to_csv('submission.csv', index = False)

In [37]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            box_off_num   R-squared:                       0.317
Model:                            OLS   Adj. R-squared:                  0.312
Method:                 Least Squares   F-statistic:                     69.00
Date:                Sun, 13 Aug 2023   Prob (F-statistic):           5.53e-48
Time:                        08:07:09   Log-Likelihood:                -9387.8
No. Observations:                 600   AIC:                         1.879e+04
Df Residuals:                     595   BIC:                         1.881e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     -1.83e+06   4.09e+05     -4.472      0.000   -2.63e+06   -1.03e+06
time          1.619e+04   4395.291      3.683      0.000    7556.523    2.48e+04
dir_prev_num  2.224e+04   5.87e+04      0.379      0.705    -9.3e+04    1.38e+05
num_staff     4781.5419    510.203      9.372      0.000    3779.524    5783.560
num_actor     4.403e+04   2.55e+04      1.727      0.085   -6029.166    9.41e+04
==============================================================================
Omnibus:                      511.565   Durbin-Watson:                   2.028
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            11346.125
Skew:                           3.769   Prob(JB):                         0.00
Kurtosis:                      22.926   Cond. No.                     1.57e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.57e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""